In [ ]:
pip install scikit-optimize

In [2]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['axes.grid'] = True
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']
import pandas as pd
import numpy as np
import seaborn as sns
import operator
import timeit
import scipy.stats as st
from joblib import Parallel, delayed
import scipy.stats as stats
from concurrent.futures import ThreadPoolExecutor, as_completed
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from scipy.optimize import dual_annealing
import warnings
warnings.filterwarnings('ignore')

In [3]:
summary = {
    'Purchase Cost': [12, 7, 6, 37],
    'Lead Time': [9, 6, 15, 12],
    'Size': [0.57, 0.05, 0.53, 1.05],
    'Selling Price': [16.10, 8.60, 10.20, 68],
    'Starting Stock': [2750, 22500, 5200, 1400],
    'Ch': [0.2 * 12, 0.2 * 7, 0.2 * 6, 0.2 * 37],
    'Co': [1000, 1200, 1000, 1200],
    'Probability': [0.76, 1.00, 0.70, 0.23],
    'Mean Demand (Lead Time)': [103.50, 648.55, 201.68, 150.06],
    'Std. Dev. of Demand (Lead Time)': [37.32, 26.45, 31.08, 3.21],
    'Expected Demand (Lead Time)': [705, 3891, 2266, 785],
    'Annual demand': [28670, 237370, 51831, 13056]
}

In [4]:
class Product:
    def __init__(self, i):
        """
        :type i: int - Product number
        """
        self.i = i
        self.unit_cost = summary['Purchase Cost'][i - 1]
        self.lead_time = summary['Lead Time'][i - 1]
        self.size = summary['Size'][i - 1]
        self.selling_price = summary['Selling Price'][i - 1]
        self.holding_cost = summary['Ch'][i - 1]
        self.ordering_cost = summary['Co'][i - 1]
        self.probability = summary['Probability'][i - 1]
        self.starting_stock = summary['Starting Stock'][i - 1]
        self.demand_lead = summary['Expected Demand (Lead Time)'][i - 1]

        mean_demand = summary['Mean Demand (Lead Time)'][i - 1]
        std_dev_demand = summary['Std. Dev. of Demand (Lead Time)'][i - 1]
        self.mean = np.log(mean_demand) if mean_demand > 0 else 0
        self.sd = np.log(std_dev_demand) if std_dev_demand > 0 else 0

def daily_demand(mean, sd, probability):
    random_num = np.random.uniform(0, 1)
    if random_num > probability:
        return 0
    else:
        return np.exp(np.random.normal(mean, sd))

def MCS(product, q, r):
    inventory = product.starting_stock
    mean = product.mean
    sd = product.sd
    lead_time = product.lead_time
    probability = product.probability

    order_placed = False
    order_time = 0
    stock_out = 0

    data = {'inv_level': [], 'daily_demand': [], 'units_sold': [], 'units_lost': [], 'orders': []}

    for day in range(1, 365):
        day_demand = daily_demand(mean, sd, probability)
        data['daily_demand'].append(day_demand)

        if inventory <= r and not order_placed:
            order_placed = True
            order_time = day

        if order_placed and (day-order_time) == lead_time:
            data['orders'].append(q)
            inventory += q
            order_placed = False
            order_time = 0

        if inventory - day_demand >= 0:
            data['units_sold'].append(day_demand)
            inventory -= day_demand
        elif inventory - day_demand < 0:
            data['units_sold'].append(inventory)
            data['units_lost'].append(day_demand - inventory)
            inventory = 0
            stock_out += 1

        data['inv_level'].append(inventory)

    return data

def profit_calculation(data, product):
    unit_cost = product.unit_cost
    selling_price = product.selling_price
    holding_cost = product.holding_cost
    order_cost = product.ordering_cost
    size = product.size
    days = 365

    revenue = sum(data['units_sold']) * selling_price
    Co = len(data['orders']) * order_cost
    Ch = sum(data['inv_level']) * holding_cost * size / days
    cost = sum(data['orders']) * unit_cost

    profit = revenue - cost - Co - Ch

    return profit

def simulation(product, q, r, num_simulations = 50):
    profit_list = []
    orders_lost_list = []
    for sim in range(num_simulations):
        data = MCS(product, q, r)

        profit = profit_calculation(data, product)
        profit_list.append(profit)

        total_demand = sum(data['daily_demand'])
        unsold_orders = sum(data['units_lost'])
        orders_lost_list.append(unsold_orders/total_demand)

    return profit_list, orders_lost_list

def ContinuousReview(product, q_guess, r_guess):
    q_low = q_guess - 1000
    q_high = q_guess + 1000
    q_range = [i for i in range(int(q_low), int(q_high), 10)]

    r_low = r_guess - 300
    r_high = r_guess + 300
    r_range = [i for i in range(int(r_low), int(r_high), 10)]

    review_dict = {}

    for q in q_range:
        for r in r_range:
            p_list, o_list, _ = simulation(product, q, r)
            review_dict[(q, r)] = (
                np.mean(p_list), np.quantile(p_list, 0.05), np.quantile(p_list, 0.95), np.std(p_list), np.mean(o_list))

    return review_dict

## scikit optimize

In [5]:
spaces = {
    1: [Integer(2250, 4000, name='q'), Integer(4000, 4500, name='r')],
    2: [Integer(20000, 25000, name='q'), Integer(30000, 35000, name='r')],
    3: [Integer(5000, 10000, name='q'), Integer(7000, 8000, name='r')],
    4: [Integer(1000, 2000, name='q'), Integer(2000, 3000, name='r')]
}

def objective(q, r, product_num):
    product = Product(product_num)
    profit_list, _ = simulation(product, q, r, num_simulations=50)
    return -np.mean(profit_list)  # We minimize the negative profit to maximize profit


total_profit = 0
for i in range(1, 5):
    space = spaces[i]

    @use_named_args(space)
    def objective_with_product(q, r):
        return objective(q, r, i)

    res = gp_minimize(objective_with_product, space, n_calls=50, random_state=0)
    optimal_q = res.x[0]
    optimal_r = res.x[1]
    expected_profit = -res.fun
    total_profit += expected_profit

    print(f"Product {i} - q: {optimal_q}, r: {optimal_r}, Profit: {expected_profit:.2f}")

print(f"Total profit → {total_profit:.2f}")

Product 1 - q: 3959, r: 4414, Profit: 527903.54
Product 2 - q: 25000, r: 35000, Profit: 1963340.47
Product 3 - q: 10000, r: 8000, Profit: 759931.41
Product 4 - q: 1855, r: 2021, Profit: 662572.55
Total profit → 3913747.97


In [7]:
def simulation(product, q, r, num_simulations=50):
    profit_list = []
    safety_stock_list = []
    lost_order_proportion_list = []

    for sim in range(num_simulations):
        data = MCS(product, q, r)

        profit = profit_calculation(data, product)
        profit_list.append(profit)

        total_demand = sum(data['daily_demand'])
        unsold_orders = sum(data['units_lost'])
        lost_order_proportion_list.append(unsold_orders / total_demand)

        safety_stock = np.mean([max(0, r - daily_demand(product.mean, product.sd, product.probability)) for _ in range(100)])
        safety_stock_list.append(safety_stock)

    return {
        'profit': profit_list,
        'mean_profit': np.mean(profit_list),
        'std_profit': np.std(profit_list),
        '5th_percentile_profit': np.percentile(profit_list, 5),
        '95th_percentile_profit': np.percentile(profit_list, 95),
        'mean_safety_stock': np.mean(safety_stock_list),
        'lost_order_proportion': np.mean(lost_order_proportion_list)
    }

spaces = {
    1: [Integer(2250, 4000, name='q'), Integer(4000, 4500, name='r')],
    2: [Integer(20000, 25000, name='q'), Integer(30000, 35000, name='r')],
    3: [Integer(5000, 10000, name='q'), Integer(7000, 8000, name='r')],
    4: [Integer(1000, 2000, name='q'), Integer(2000, 3000, name='r')]
}

def objective(q, r, product_num):
    product = Product(product_num)
    sim_results = simulation(product, q, r, num_simulations=50)
    return -sim_results['mean_profit']  # Minimize negative profit to maximize profit

total_profit = 0
metrics = []

for i in range(1, 5):
    space = spaces[i]

    @use_named_args(space)
    def objective_with_product(q, r):
        return objective(q, r, i)

    res = gp_minimize(objective_with_product, space, n_calls=50, random_state=0)
    optimal_q = res.x[0]
    optimal_r = res.x[1]

    # Get simulation results for optimal (q, r)
    product = Product(i)
    sim_results = simulation(product, optimal_q, optimal_r, num_simulations=50)

    metrics.append({
        'Product': i,
        'q': optimal_q,
        'r': optimal_r,
        'μ_profit': sim_results['mean_profit'],
        'σ_profit': sim_results['std_profit'],
        '5th_percentile_profit': sim_results['5th_percentile_profit'],
        '95th_percentile_profit': sim_results['95th_percentile_profit'],
        'μ_SS': sim_results['mean_safety_stock']
      })

    total_profit += sim_results['mean_profit']

print(f"Total profit: {total_profit:.2f}")
for metric in metrics:
    print(f"Product {metric['Product']}:")
    print(f"q: {metric['q']}, r: {metric['r']}")
    print(f"μ_profit: {metric['μ_profit']:.2f}")
    print(f"σ_profit: {metric['σ_profit']:.2f}")
    print(f"5th percentile profit: {metric['5th_percentile_profit']:.2f}")
    print(f"95th percentile profit: {metric['95th_percentile_profit']:.2f}")
    print(f"μ_SS: {metric['μ_SS']:.2f}")


Total profit: 3884802.49
Product 1:
q: 3945, r: 4430
μ_profit: 521836.49
σ_profit: 34849.75
5th percentile profit: 450067.98
95th percentile profit: 573410.25
μ_SS: 3686.75
Product 2:
q: 25000, r: 35000
μ_profit: 1958021.72
σ_profit: 109806.55
5th percentile profit: 1775876.36
95th percentile profit: 2132772.71
μ_SS: 27905.73
Product 3:
q: 10000, r: 7956
μ_profit: 771964.35
σ_profit: 48815.27
5th percentile profit: 689908.32
95th percentile profit: 830861.59
μ_SS: 6647.05
Product 4:
q: 1912, r: 2848
μ_profit: 632979.93
σ_profit: 125500.15
5th percentile profit: 427446.34
95th percentile profit: 788737.67
μ_SS: 2781.68
